In [226]:
import gurobipy as gp
from gurobipy import GRB
import random as rd
import requests
import folium
import osmnx as ox
import warnings

In [227]:
A = [0,1,2,3,4,5,6,7,8,9,10]
T = [0,1,2,3,4,5]
V = ['walk', 'bike', 'car']
shared = [False, False, True]
Available = [1000, 10, 5]

In [228]:
Q = 15
Budget = 100
n_passengers = 1
cost_rate_v = {'walk': 0, 'bike': 1.0, 'car': 2.0}
travel_time_v = {'walk': 1, 'bike': 0.5, 'car': 0.15}
fixed_cost = 5

# coordinates = matrix of size len(A)

coordinates = {
    0: (43.651070, -79.347015),  # Toronto City Hall (Depot)
    1: (43.6677, -79.3949),      # Royal Ontario Museum
    2: (43.6426, -79.3871),      # CN Tower
    3: (43.7365, -79.3845),      # Ontario Science Centre
    4: (43.7111, -79.4115),      # Casa Loma
    5: (43.6534, -79.3725),      # St. Lawrence Market
    6: (43.7267, -79.6137),      # Rouge National Urban Park
    7: (43.7396, -79.5461),      # High Park
    8: (43.7731, -79.5207),      # Toronto Zoo
    9: (43.7714, -79.2579),      # Scarborough Town Centre
    10: (43.6536, -79.3923),     # Art Gallery of Ontario (AGO)
}

def fetch_osrm_matrix(coords, profile='car'):
    node_ids = sorted(coords.keys())
    coords_str = ";".join(f"{coords[i][1]},{coords[i][0]}" for i in node_ids)
    url = f"http://router.project-osrm.org/table/v1/{profile}/{coords_str}?annotations=distance,duration"
    response = requests.get(url)
    data = response.json()
    
    if 'distances' not in data or 'durations' not in data:
        raise Exception(f"OSRM API error or rate limit exceeded: {data}")
    
    distances = data['distances']
    durations = data['durations']
    dist_dict = {}
    dur_dict = {}
    
    n = len(node_ids)
    for i_idx in range(n):
        for j_idx in range(n):
            i = node_ids[i_idx]
            j = node_ids[j_idx]
            dist_dict[(i,j)] = distances[i_idx][j_idx]
            dur_dict[(i,j)] = durations[i_idx][j_idx]
    return dist_dict, dur_dict

distance_ij_car, duration_ij_car = fetch_osrm_matrix(coordinates, profile='car')
distance_ij_walk, duration_ij_walk = fetch_osrm_matrix(coordinates, profile='foot')

distance_ij = {}
t_ij_v = {}

for v in V:
    if v == 'car':
        dist = distance_ij_car
        dur = duration_ij_car
    if v == 'bike':
        dist = distance_ij_car
        dur = duration_ij_car
    elif v == 'walk':
        dist = distance_ij_walk
        dur = duration_ij_walk
    else:
        dist = {}
        dur = {}

    for i in A:
        for j in A:
            distance_ij[(i,j)] = dist.get((i,j), 0)
            t_ij_v[(v,i,j)] = dur.get((i,j), 0) / 60

print(t_ij_v)

tv_j = {
    0: 0, # Depot usually has 0 service time
    1: 2,
    2: 2,
    3: 2,
    4: 1.5,
    5: 1,
    6: 2,
    7: 2,
    8: 3,
    9: 1.5,
    10: 2,
}

open_windows = {
    0: (8, 18), # Depot: Open all day 
    1: (10, 17),
    2: (9, 22),
    3: (10, 17),
    4: (8, 20),
    5: (8, 18),
    6: (7, 20),
    7: (6, 23),
    8: (9, 17),
    9: (10, 21),
    10: (10, 18),
}

for i in A:
    print(f"open_hours({i}) = {open_windows[i][0]} to {open_windows[i][1]}")


{('walk', 0, 0): 0.0, ('walk', 0, 1): 13.983333333333333, ('walk', 0, 2): 5.671666666666667, ('walk', 0, 3): 19.226666666666667, ('walk', 0, 4): 19.926666666666666, ('walk', 0, 5): 5.036666666666666, ('walk', 0, 6): 28.001666666666665, ('walk', 0, 7): 32.065000000000005, ('walk', 0, 8): 33.54666666666667, ('walk', 0, 9): 22.025, ('walk', 0, 10): 8.203333333333333, ('walk', 1, 0): 12.998333333333333, ('walk', 1, 1): 0.0, ('walk', 1, 2): 8.875, ('walk', 1, 3): 20.488333333333333, ('walk', 1, 4): 13.856666666666666, ('walk', 1, 5): 8.933333333333334, ('walk', 1, 6): 31.638333333333332, ('walk', 1, 7): 29.586666666666666, ('walk', 1, 8): 29.323333333333334, ('walk', 1, 9): 28.513333333333332, ('walk', 1, 10): 5.6899999999999995, ('walk', 2, 0): 5.556666666666667, ('walk', 2, 1): 9.793333333333333, ('walk', 2, 2): 0.0, ('walk', 2, 3): 20.826666666666664, ('walk', 2, 4): 16.03666666666667, ('walk', 2, 5): 4.895, ('walk', 2, 6): 25.098333333333336, ('walk', 2, 7): 29.16166666666667, ('walk', 

In [229]:
m = gp.Model("model")
# Variables
x = m.addVars(A, A, V, T, vtype=GRB.BINARY, name="x")

m.setObjective(gp.quicksum(x[i, j, v, t]  for i in A for j in A for v in V for t in T), GRB.MAXIMIZE) 
# Constraints

m.addConstrs(gp.quicksum(x[i, i, v, t] for t in T for v in V) == 0 for i in A)
m.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for v in V) == gp.quicksum(x[j, i, v, t] for i in A for v in V) for j in A for t in T)
m.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for t in T for v in V) <= 1 for j in A[1:])

m.addConstrs(gp.quicksum(x[0, j, v, t] for j in A[1:] for v in V) <= 1 for t in T )

m.addConstrs(gp.quicksum(x[i, j, v, t] * (tv_j[j] + t_ij_v[v, i, j]) for i in A for j in A for v in V ) <= Q for t in T )

#Miller-Tucker-Zemlin formulation Subtour Elimination Constraints
q = [rd.randint(3, 3) for i in range(len(A)-1)]  # generate the random vector
u = m.addVars([i for i in range(len(A)-1)],V, vtype=GRB.CONTINUOUS, name="u")
for v in V: u[0, v].LB = 0
for v in V: u[0, v].UB = 0
P = [i for i in range(len(A)-1)]
for v in V :
    for i in P :
            
    #             u[i].LB = q[i]
        u[i, v].LB = min([(q[i]+(tv_j[i] + t_ij_v[v, j,i])) for j in P for t in T])
        u[i, v].UB = Q
    #     print([(q[i]+tm[j][i])* xijt[i,j,t] for i in P for j in P for t in T])
        m.addConstrs( u[i, v] - u[j, v] + Q * x[i, j, v, t] <= Q - q[j] for i in P for j in P for t in T if j != 0 )

In [230]:
m.optimize()
if m.status == GRB.OPTIMAL:
    print("Optimal solution found")
    for v in V:
        for i in A:
            for j in A:
                for t in T:
                    if x[i, j, v, t].x > 0.5:
                        print(f"Vehicle {v} travels from {i} to {j} at day {t}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16299 rows, 2208 columns and 53778 nonzeros
Model fingerprint: 0x9044a3c0
Variable types: 30 continuous, 2178 integer (2178 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 15691 rows and 1365 columns
Presolve time: 0.05s
Presolved: 608 rows, 843 columns, 4896 nonzeros
Variable types: 15 continuous, 828 integer (828 binary)
Found heuristic solution: objective 14.0000000

Root relaxation: objective 1.600000e+01, 223 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time


In [231]:
m1 = gp.Model("model tw")

# Variables
x = m1.addVars(A, A, V, T, vtype=GRB.BINARY, name="x")
t_ = m1.addVars(A,V, T, vtype=GRB.CONTINUOUS, name="t")
y = m1.addVars(T, vtype=GRB.BINARY, name='y')
b = m1.addVars(T, vtype=GRB.CONTINUOUS, name='b')

# Obj Function
m1.setObjective(gp.quicksum(x[i, j, v, t] * (1 / t_ij_v[v, i, j]) for i in A for j in A for v in V for t in T if i != j and t_ij_v[v, i, j] > 1e-6), GRB.MAXIMIZE)

# Constraints
m1.addConstrs(gp.quicksum(x[i, i, v, t] for t in T for v in V) == 0 for i in A)
m1.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for v in V) == gp.quicksum(x[j, i, v, t] for i in A for v in V) for j in A for t in T)
m1.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for t in T for v in V) <= 1 for j in A[1:])
m1.addConstrs(gp.quicksum(x[0, j, v, t] for j in A[1:] for v in V) == y[t] for t in T)
m1.addConstrs(gp.quicksum(x[i, j, v, t] * (tv_j[j] + t_ij_v[v, i, j]) for i in A for j in A for v in V ) <= Q for t in T )

m1.addConstrs(y[t] <= y[t+1] for t in T[:-1])  # Ensure y is non-decreasing
m1.addConstr(y[0]==1)  # Ensure y is non-increasing

# Time window constraints
m1.addConstrs(t_[i, v, t] >= open_windows[i][0] for i in A for v in V for t in T)
m1.addConstrs(t_[i, v, t] <= open_windows[i][1] for i in A for v in V for t in T)

# Daily budget constraints
m1.addConstrs(gp.quicksum((n_passengers if shared[V.index(v)] == False else 1)*x[i, j, v, t] * cost_rate_v[v]* t_ij_v[v, i, j] for i in A for j in A for v in V ) <= b[t] for t in T)
m1.addConstr(gp.quicksum(b[t] for t in T) + fixed_cost<= Budget)

#m1.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for j in A) <= Available[V.index(v)] for v in V for t in T)
m1.addConstrs( x[i, j, v, t] * n_passengers  <= Available[V.index(v)] for i in A for j in A for v in V for t in T)
m1.addConstrs(t_[i, v, t] + tv_j[i] + t_ij_v[v, i, j] - 1000 * (1 - x[i, j, v, t]) <= t_[j, v, t] for v in V for i in A for j in A[1:] for t in T   )

#Miller-Tucker-Zemlin formulation Subtour Elimination Constraints
q = [rd.randint(3, 3) for i in range(len(A)-1)]  # generate the random vector
u = m1.addVars([i for i in range(len(A)-1)],V, vtype=GRB.CONTINUOUS, name="u")
for v in V: u[0, v].LB = 0
for v in V: u[0, v].UB = 0
P = [i for i in range(len(A)-1)]
for v in V :
    for i in P :
            
    #             u[i].LB = q[i]
        u[i, v].LB = min([(q[i]+(tv_j[i] + t_ij_v[v, j,i])) for j in P for t in T])
        u[i, v].UB = Q
    #     print([(q[i]+tm[j][i])* xijt[i,j,t] for i in P for j in P for t in T])
        m1.addConstrs( u[i, v] - u[j, v] + Q * x[i, j, v, t] <= Q - q[j] for i in P for j in P for t in T if j != 0 )

In [232]:
m1.optimize()
if m1.status == GRB.OPTIMAL:
    print("Optimal solution found")
    for t in T:
        for i in A:
            for j in A:
                for v in V:
                    if x[i, j, v, t].x > 0.5:
                        print(f"Vehicle {v} travels from {i} to {j} at day {t}")
                        print(f"Cost for vehicle {v} is {(n_passengers if shared[V.index(v)] == False else 1)*cost_rate_v[v] * t_ij_v[v, i, j]}")
                        print(f"Time at {i} is {t_[i, v, t].x}")
                        print(f"Time at {j} is {t_[j, v, t].x}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20866 rows, 2418 columns and 62621 nonzeros
Model fingerprint: 0xd77d7c60
Variable types: 234 continuous, 2184 integer (2184 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e-02, 2e-01]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 1e+03]
Presolve removed 19646 rows and 1483 columns
Presolve time: 0.05s
Presolved: 1220 rows, 935 columns, 12735 nonzeros
Variable types: 109 continuous, 826 integer (825 binary)

Root relaxation: objective 1.183415e+00, 219 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.18342    0    8          -    1.18342      -     -    0s
H    0     0  

In [246]:
trips = {t: [] for t in T}

if m1.status == GRB.OPTIMAL:
    print("Optimal solution found:\n")

    for t in T:
        print(f"--- Trips for Day {t} ---")
        trips_found = False
        for i in A:
            for j in A:
                for v in V:
                    if i != j and x[i, j, v, t].x > 0.5:
                        trips_found = True
                        trips[t].append((v, i, j))
                        print(f"Vehicle [{v}] from {i} to {j}")
        if not trips_found:
            print("No trips on this day.")
        
        print("\n")

Optimal solution found:

--- Trips for Day 0 ---
Vehicle [bike] from 0 to 10
Vehicle [car] from 10 to 0


--- Trips for Day 1 ---
Vehicle [car] from 0 to 8
Vehicle [car] from 7 to 0
Vehicle [walk] from 8 to 7


--- Trips for Day 2 ---
Vehicle [car] from 0 to 9
Vehicle [car] from 9 to 0


--- Trips for Day 3 ---
Vehicle [bike] from 0 to 5
Vehicle [walk] from 5 to 0


--- Trips for Day 4 ---
Vehicle [bike] from 0 to 2
Vehicle [walk] from 2 to 0


--- Trips for Day 5 ---
Vehicle [car] from 0 to 1
Vehicle [walk] from 1 to 0




In [245]:
warnings.filterwarnings("ignore", category=FutureWarning, message=".*folium.*")

def generate_day_colors(num_days):
        colors = []
        for _ in range(num_days):
            color = f"#{rd.randint(0, 150):02x}{rd.randint(0, 150):02x}{rd.randint(0, 150):02x}"
            colors.append(color)        
        return colors

if m1.status == GRB.OPTIMAL:
    G = ox.graph_from_place('Toronto, Ontario, Canada', network_type='drive')
    
    mymap = folium.Map(location=coordinates[0], zoom_start=11)
    
    day_colors = generate_day_colors(len(trips))
    
    def snap_to_nearest_road(graph, coords):
        return ox.distance.nearest_nodes(graph, X=coords[1], Y=coords[0])
    
    for node, coords in coordinates.items():
        icon_color = 'green' if node == 0 else 'blue'  # Green for depot (node 0), Blue for others
        folium.Marker(
            location=coords,
            popup=f"Node {node}",
            icon=folium.Icon(color=icon_color, icon='info-sign')
        ).add_to(mymap)
    
    for t, trip_list in trips.items():
        route_color = day_colors[t]
        for v, i, j in trip_list:
            node_i = snap_to_nearest_road(G, coordinates[i])
            node_j = snap_to_nearest_road(G, coordinates[j])
    
            try:
                route = ox.shortest_path(G, node_i, node_j, weight='length')
                popup_text = f"Day: {t}, Vehicle: {v}, From {i} to {j}"
    
                ox.plot_route_folium(
                    G, route, route_map=mymap, color=route_color, weight=5, opacity=0.7
                )
    
                folium.PolyLine(
                    locations=[(G.nodes[n]['y'], G.nodes[n]['x']) for n in route],
                    color=route_color,
                    weight=5,
                    opacity=0.7,
                    popup=popup_text
                ).add_to(mymap)
    
            except nx.NetworkXNoPath:
                print(f"No path between nodes {i} and {j} for Day {t}, Vehicle {v}")
    
    latitudes = [coords[0] for coords in coordinates.values()]
    longitudes = [coords[1] for coords in coordinates.values()]
    
    min_lat = min(latitudes)
    max_lat = max(latitudes)
    min_lon = min(longitudes)
    max_lon = max(longitudes)
    
    mymap.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    
    mymap.save("routing_snapped.html")
    print("Map saved as routing_snapped.html")

else:
    print("Model not solved optimally. Cannot plot routes.")
    
if m1.status == GRB.OPTIMAL:
    mymap = folium.Map(location=coordinates[0], zoom_start=13)

    for node, coords in coordinates.items():
        icon_color = 'green' if node == 0 else 'blue'  # Green for depot (node 0), Blue for others
        folium.Marker(
            location=coords,
            popup=f"Node {node}",
            icon=folium.Icon(color=icon_color, icon='info-sign')
        ).add_to(mymap)

    day_colors = generate_day_colors(len(T))

    for t in T:
        route_color = day_colors[t]
        for v in V:
            for i in A:
                for j in A:
                    if i != j and x[i, j, v, t].X > 0.5:
                        folium.PolyLine(
                            locations=[coordinates[i], coordinates[j]],
                            color=route_color,
                            weight=5,
                            opacity=0.7,
                            popup=f"Day: {t}, Vehicle: {v}, From {i} to {j}"
                        ).add_to(mymap)

    mymap.save("routing.html")
    print("Map saved as routing.html")

else:
    print("Model not solved optimally. Cannot plot routes.")

Map saved as routing_snapped.html
Map saved as routing.html
